In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sys
import datetime
import os
import h5py

import random
import wandb
import torch

/opt/homebrew/Caskroom/miniforge/base/envs/me555/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
output1 = pd.read_csv('../Data/output/output_1.csv')
input_df1 = output1.groupby('i').agg({'tHin_i': 'first', 'tHout_i': 'first', 'SOC': 'first'})

In [18]:

path = '../Data/scenarios'
all_files = os.listdir(path)

# sort files according to last digit
all_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

li = []
for filename in all_files:
    # no header, no index
    print(filename)
    df = pd.read_csv(path + '/' + filename, header=None, index_col=None)
    li.append(df)
solar_df = pd.concat(li, axis=0, ignore_index=True)

scenarios_1.csv
scenarios_2.csv
scenarios_3.csv
scenarios_4.csv
scenarios_5.csv
scenarios_6.csv
scenarios_7.csv
scenarios_8.csv
scenarios_9.csv
scenarios_10.csv
scenarios_11.csv
scenarios_12.csv
scenarios_13.csv
scenarios_14.csv
scenarios_15.csv
scenarios_16.csv
scenarios_17.csv
scenarios_18.csv
scenarios_19.csv
scenarios_20.csv
scenarios_21.csv
scenarios_22.csv
scenarios_23.csv
scenarios_24.csv
scenarios_25.csv
scenarios_26.csv
scenarios_27.csv
scenarios_28.csv
scenarios_29.csv
scenarios_30.csv


In [19]:
nScenarios = solar_df.shape[0]
nTimeSteps = solar_df.shape[1]
nEVs = input_df1.shape[0]

print('nScenarios: ', nScenarios)
print('nTimeSteps: ', nTimeSteps)
print('nEVs: ', nEVs)


nScenarios:  3000
nTimeSteps:  48
nEVs:  300


In [20]:
# TOU prices
nbTime = 48
off_peak = 0.281
mid_peak = 0.357
peak = 0.584

peak_times = [
    (0, 16, off_peak),
    (16, 22, mid_peak),
    (22, 24, peak),
    (24, 28, mid_peak),
    (28, 34, peak),
    (34, 40, mid_peak),
    (40, 48, off_peak),
]

TOUPurchasePrice = [0.0] * nbTime
TOUSellPrice = [0.0] * nbTime

for t in range(nbTime):
    for start, end, price in peak_times:
        if start <= t < end:
            TOUPurchasePrice[t] = price
            TOUSellPrice[t] = 0.9 * price
            break


In [21]:
# expand solar_df t0 [nScenarios, nTimeSteps ,1]
# solar_array = solar_df.values.reshape(nScenarios, nTimeSteps, 1)
solar_array = solar_df.values

TOUPurchasePrice = np.array(TOUPurchasePrice).reshape(1, nTimeSteps)

# repeat TOU purchase price for nScenarios scenarios
TOUPurchasePrice = np.repeat(TOUPurchasePrice, nScenarios, axis=0)

In [22]:
TOUPurchasePrice.shape

(3000, 48)

In [23]:
input_df1

,tHin_i,tHout_i,SOC
i,,,
0,22,32,0.521269
1,14,34,0.506830
2,22,34,0.538214
3,26,34,0.490184
4,14,24,0.534222
...,...,...,...
295,18,34,0.542584
296,20,36,0.660097
297,26,36,0.469156


In [24]:
TOUPurchasePrice.shape

(3000, 48)

In [25]:
input_df1

,tHin_i,tHout_i,SOC
i,,,
0,22,32,0.521269
1,14,34,0.506830
2,22,34,0.538214
3,26,34,0.490184
4,14,24,0.534222
...,...,...,...
295,18,34,0.542584
296,20,36,0.660097
297,26,36,0.469156


In [26]:
#  for each EV in input_df1, make tHin_i and tHout_i as one-hot vectors and make 2D array

Tin_onehot = np.zeros((input_df1.shape[0], nTimeSteps))
Tout_onehot = np.zeros((input_df1.shape[0], nTimeSteps))

for i in range(input_df1.shape[0]):
    Tin_onehot[i, input_df1['tHin_i'].iloc[i]] = 1
    Tout_onehot[i, input_df1['tHout_i'].iloc[i]] = 1


# perform element-wise OR for numpy arrays
Tin_out_onehot = np.zeros(Tin_onehot.shape)
for i in range(Tin_onehot.shape[0]):
    Tin_out_onehot[i, :] = np.logical_or(Tin_onehot[i], Tout_onehot[i])

In [27]:
Tin_out_onehot.shape

(300, 48)

In [28]:
# repeat Tin_out_onehot for nScenarios scenarios
Tin_out_onehot_data = np.repeat(Tin_out_onehot.reshape(1, nEVs, nTimeSteps), nScenarios, axis=0)

# repeat solar_array for nEVs EVs
# solar_array_data = np.repeat(solar_array.reshape(nScenarios, -1, nTimeSteps), nEVs, axis=1)
solar_array_data = solar_array.copy()

In [29]:
TOUPurchasePrice.shape

(3000, 48)

In [30]:
# repeat TOUPurchasePrice for nEVs EVs
# TOUPurchasePrice_data = np.repeat(TOUPurchasePrice.reshape(nScenarios, -1, nTimeSteps), nEVs, axis=1)
TOUPurchasePrice_data = TOUPurchasePrice.copy()

# repeat SOC for nScenarios scenarios and expand to [nScenarios, nEVs, 1]
# SOC_data = np.repeat(input_df1['SOC'].values.reshape(1, nEVs), nScenarios, axis=0).reshape(nScenarios, nEVs, 1)
SOC_data = np.repeat(input_df1['SOC'].values.reshape(1, nEVs), nScenarios, axis=0)

In [31]:
print("solar array shape: ", solar_array_data.shape)
print("Tin_out_onehot shape: ", Tin_out_onehot_data.shape)
print("TOUPurchasePrice shape: ", TOUPurchasePrice_data.shape)
print("SOC shape: ", SOC_data.shape)

solar array shape:  (3000, 48)
Tin_out_onehot shape:  (3000, 300, 48)
TOUPurchasePrice shape:  (3000, 48)
SOC shape:  (3000, 300)


In [32]:
Tin_out_onehot_data_reshaped = Tin_out_onehot_data.reshape(nScenarios, -1)

In [33]:
Tin_out_onehot_data_reshaped.shape

(3000, 14400)

In [36]:
# concatenate all data
input_data = np.concatenate((solar_array_data, Tin_out_onehot_data_reshaped, TOUPurchasePrice_data, SOC_data), axis=1)

### Now create output data

In [37]:
input_data.shape

(3000, 14796)

In [38]:
path = '../Data/output'
all_files = os.listdir(path)

# sort files according to last digit
all_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

li = []
for filename in all_files:
    # no header, no index
    print(filename)
    df = pd.read_csv(path + '/' + filename, index_col=None)

    df.drop(['tHin_i', 'tHout_i', 'SOC', 'DayAheadUtilityPowerOutput', 'DayAheadChargingPower'], axis=1, inplace=True)

    li.append(df)
# output_df = pd.concat(li, axis=0, ignore_index=True)

output_1.csv
output_2.csv
output_3.csv
output_4.csv
output_5.csv
output_6.csv
output_7.csv
output_8.csv
output_9.csv
output_10.csv
output_11.csv
output_12.csv
output_13.csv
output_14.csv
output_15.csv
output_16.csv
output_17.csv
output_18.csv
output_19.csv
output_20.csv
output_21.csv
output_22.csv
output_23.csv
output_24.csv
output_25.csv
output_26.csv
output_27.csv
output_28.csv
output_29.csv
output_30.csv


In [41]:
def create_output(df_list):

    OnOff_output_data = np.zeros((nScenarios, nEVs, nTimeSteps))
    BuySell_output_data = np.zeros((nScenarios, nTimeSteps))

    for i in range(len(df_list)):
        df = df_list[i]
        scaler = i * 100

        # do not modify original df in df_list
        tmp_df = df.copy()
        tmp_df['Scenario'] = tmp_df['Scenario'] + scaler

        tmp_numpy = tmp_df.to_numpy()

        # print(tmp_numpy[:, 0].max())
        for j in range(len(tmp_numpy)):
            scenario = tmp_numpy[j, 0]

            # if(j % 100 == 0):
            #     print("scenario is: ", scenario)

            time = tmp_numpy[j, 1]
            ev = tmp_numpy[j, 2]
            buysellstatus = tmp_numpy[j, 3]
            onoffchargingstatus = tmp_numpy[j, 4]

            OnOff_output_data[scenario, ev, time] = onoffchargingstatus
            BuySell_output_data[scenario, time] = buysellstatus

    OnOff_output_data_reshaped = OnOff_output_data.reshape(nScenarios, -1)
    output = np.concatenate((OnOff_output_data_reshaped, BuySell_output_data), axis=1)

    return output

In [42]:
output = create_output(li)

In [43]:
li[-1].head()

,Scenario,Time,i,DayAheadBuySellStatus,DayAheadOnOffChargingStatus
0,0,0,0,1,0
1,0,0,1,1,0
2,0,0,2,1,0
3,0,0,3,1,0
4,0,0,4,1,0


In [44]:
# save input_data and output_data to hdf5 file
with h5py.File('../Data/raw_data.hdf5', 'w') as f:
    f.create_dataset('input_data', data=input_data)
    f.create_dataset('output_data', data=output)

In [46]:
# get device
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

PROJECT_NAME = 'EVCharging'

# Set the random seeds to improve reproducibility by removing stochasticity
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False # Force cuDNN to use a consistent convolution algorithm
    torch.backends.cudnn.deterministic = True # Force cuDNN to use deterministic algorithms if available
    torch.use_deterministic_algorithms(True) # Force torch to use deterministic algorithms if available

set_seeds(0)

# for deterministic pytorch algorithms, enable reproducibility.
os.environ['CUBLAS_WORKSPACE_CONFIG']= ":4096:8"

In [47]:
config = {
    'train_val_split': [0.80, 0.20], # These must sum to 1.0
    'batch_size' : 32, # Num samples to average over for gradient updates
    'EPOCHS' : 200, # Num times to iterate over the entire dataset
    'LEARNING_RATE' : 1e-3, # Learning rate for the optimizer
    'BETA1' : 0.9, # Beta1 parameter for the Adam optimizer
    'BETA2' : 0.999, # Beta2 parameter for the Adam optimizer
    'WEIGHT_DECAY' : 1e-4, # Weight decay parameter for the Adam optimizer
    'accum_iter': 8, # iterations to accumulate gradients
}

In [48]:
run = wandb.init(project=PROJECT_NAME, entity="jyyresearch", config=config)
raw_data = wandb.Artifact('EV-coord-raw_data', type='raw_data')
raw_data.add_file('../Data/raw_data.hdf5')
run.log_artifact(raw_data)
run.finish()

wandb: Currently logged in as: jinyi12 (jyyresearch). Use `wandb login --relogin` to force relogin
